In [1]:
from youtube_constants import YOUTUBE_API_ACCESS_TOKEN, PLAYLIST_ID



In [2]:
from youtube_data import get_all_video_in_playlist


In [3]:
import pandas as pd


l_respo_playlists = []
l_video_links = []
for p_id in PLAYLIST_ID.split(","):
    respo, video_links = get_all_video_in_playlist(api_key=YOUTUBE_API_ACCESS_TOKEN,
                                               playlist_id=p_id)
    l_video_links.append(pd.DataFrame(video_links))
    l_respo_playlists.append(respo)



In [9]:
df_video_playlist = pd.concat(l_video_links).reset_index(drop=True)


In [10]:
DF_PLAYLIST = df_video_playlist.drop(columns="snippet").merge(df_video_playlist["snippet"].apply(pd.Series), right_index=True, left_index=True)

In [33]:
DF_PLAYLIST["thumbnail"] = DF_PLAYLIST["thumbnails"].map(lambda x: x.get("high", {}).get("url"))

In [17]:
DF_PLAYLIST.to_csv("../outputs/playlist-sample.csv", index=False)

In [19]:

# now passing various video ids to get more information about the video

from youtube_data import get_videos_from_list, chunks
list_of_video_lists = list(chunks(l=DF_PLAYLIST["video_id"].tolist(), n=5))

In [20]:
list_resps = []
list_videos_dfs = []

for video_list in list_of_video_lists:
    # now we have a lists of lists that we can call the api since we cant pass in 50+ videos at one time
    respov, items_ = get_videos_from_list(api_key=YOUTUBE_API_ACCESS_TOKEN, video_id_list=video_list)
    list_resps.append(respov)
    list_videos_dfs.append(pd.DataFrame(items_))

big_df_videos = pd.concat(list_videos_dfs).reset_index(drop=True)
print(big_df_videos.shape[0])


179


In [21]:
DF_PLAYLIST.shape[0]

182

In [23]:
big_df_videos.shape[0]

179

In [26]:
import jsonlines
from youtube_data import get_transcript_from_video_id
l_transcripts = []
fout_path = "../outputs/playlist-videos-transcripts-sample.jsonl"

with jsonlines.open(fout_path, "w") as fout:
    
    for i, row in big_df_videos.iterrows():

        new_i = {}
        video_id = row["videoId"]
        new_i["videoId"] = video_id
        try:
            new_i["captions_raw"] = get_transcript_from_video_id(row["videoId"])
            new_i["caption_text_flag"] = True
        except Exception as error:
            print(f"An error occurred for video {video_id} @ index {i}:\n", error)
            new_i["captions_raw"] = None
            new_i["caption_text_flag"] = False
            print(new_i)

        
        fout.write(new_i)
        l_transcripts.append(new_i)

An error occurred for video DVv9DWIDp74 @ index 8:
 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=DVv9DWIDp74! This is most likely caused by:

No transcripts were found for any of the requested language codes: ('en',)

For this video (DVv9DWIDp74) transcripts are available in the following languages:

(MANUALLY CREATED)
None

(GENERATED)
 - es ("Spanish (auto-generated)")[TRANSLATABLE]

(TRANSLATION LANGUAGES)
 - ab ("Abkhazian")
 - aa ("Afar")
 - af ("Afrikaans")
 - ak ("Akan")
 - sq ("Albanian")
 - am ("Amharic")
 - ar ("Arabic")
 - hy ("Armenian")
 - as ("Assamese")
 - ay ("Aymara")
 - az ("Azerbaijani")
 - bn ("Bangla")
 - ba ("Bashkir")
 - eu ("Basque")
 - be ("Belarusian")
 - bho ("Bhojpuri")
 - bs ("Bosnian")
 - br ("Breton")
 - bg ("Bulgarian")
 - my ("Burmese")
 - ca ("Catalan")
 - ceb ("Cebuano")
 - zh-Hans ("Chinese (Simplified)")
 - zh-Hant ("Chinese (Traditional)")
 - co ("Corsican")
 - hr ("Croatian")
 - cs ("Czech")
 - da ("Danish")
 - dv

In [28]:
big_df_videos.columns

Index(['videoId', 'duration_seconds', 'duration_approx_minutes', 'viewCount',
       'likeCount', 'favoriteCount', 'commentCount'],
      dtype='object')

In [29]:
# uncomment if you want to use the file output
# add path to where your transcripts jsonl file is

# path_transcripts_jsonl = "../outputs/playlist-videos-transcripts-sample.jsonl"

# df_transcripts = pd.read_json(path_transcripts_jsonl, lines=True)


# or if you want to use the list
df_transcripts = pd.DataFrame(l_transcripts)

In [30]:

# making final merged df
merged_df = big_df_videos.merge(df_transcripts, right_on="videoId", left_on="videoId", how="outer")




In [31]:
merged_df.columns

Index(['videoId', 'duration_seconds', 'duration_approx_minutes', 'viewCount',
       'likeCount', 'favoriteCount', 'commentCount', 'captions_raw',
       'caption_text_flag'],
      dtype='object')

In [38]:
DF_PLAYLIST.shape[0]

182

In [39]:
merged_df.shape[0]

183

In [43]:
FINAL_PLAYLIST_MERGE = DF_PLAYLIST.merge(merged_df, right_on='videoId', left_on='video_id', how='outer', indicator=True)


In [45]:
FINAL_PLAYLIST_MERGE

,kind,etag,id,video_link,video_id,publishedAt,channelId,title,description,thumbnails,...,thumbnail,videoId,duration_seconds,duration_approx_minutes,viewCount,likeCount,favoriteCount,commentCount,captions_raw,caption_text_flag
0,youtube#playlistItem,sSCWVjV-x-kSUboloSx5SxV2ioA,UExUbld0Wk1UZUhyUEZzQzR0RHB6TUZlcnN6WV9FUWx2SC...,https://www.youtube.com/watch?v=V7Cm9kM8q0o,V7Cm9kM8q0o,2024-07-25T14:10:47Z,UC1c9AbJs9TxnqYFF3lk0ROg,"Wednesday, July 24, 2024 - Regular City Counci...",,{'default': {'url': 'https://i.ytimg.com/vi/V7...,...,https://i.ytimg.com/vi/V7Cm9kM8q0o/hqdefault.jpg,V7Cm9kM8q0o,4617.0,77.0,163,0,0,NaN,e I call this meeting to order R call please c...,True
1,youtube#playlistItem,5XKR_CKPghHSWzJqcCZ4zTAc2eM,UExUbld0Wk1UZUhyUEZzQzR0RHB6TUZlcnN6WV9FUWx2SC...,https://www.youtube.com/watch?v=pIcRrKVUea8,pIcRrKVUea8,2024-07-11T00:50:11Z,UC1c9AbJs9TxnqYFF3lk0ROg,"Wednesday, July 10, 2024 - Regular Council Me...",,{'default': {'url': 'https://i.ytimg.com/vi/pI...,...,https://i.ytimg.com/vi/pIcRrKVUea8/hqdefault.jpg,pIcRrKVUea8,3155.0,53.0,131,2,0,NaN,I call this meeting to order please R [Music] ...,True
2,youtube#playlistItem,AqfwLvdut1kia4hL1HDS7LnZFr4,UExUbld0Wk1UZUhyUEZzQzR0RHB6TUZlcnN6WV9FUWx2SC...,https://www.youtube.com/watch?v=pX5dVXRNMz0,pX5dVXRNMz0,2024-07-11T00:11:40Z,UC1c9AbJs9TxnqYFF3lk0ROg,"Wednesday, July 10, 2024 - Special City Counc...",,{'default': {'url': 'https://i.ytimg.com/vi/pX...,...,https://i.ytimg.com/vi/pX5dVXRNMz0/hqdefault.jpg,pX5dVXRNMz0,950.0,16.0,33,0,0,NaN,I call this meeting to order Rong call please ...,True
3,youtube#playlistItem,DwxhNRsHedLSLmHZh35yP8iTMqY,UExUbld0Wk1UZUhyUEZzQzR0RHB6TUZlcnN6WV9FUWx2SC...,https://www.youtube.com/watch?v=krXL4pRr4ow,krXL4pRr4ow,2024-07-04T00:34:46Z,UC1c9AbJs9TxnqYFF3lk0ROg,"Wednesday, July 3, 2024 - Special City Counci...",,{'default': {'url': 'https://i.ytimg.com/vi/kr...,...,https://i.ytimg.com/vi/krXL4pRr4ow/hqdefault.jpg,krXL4pRr4ow,1160.0,19.0,86,0,0,NaN,e e I call this meeting to order R call please...,True
4,youtube#playlistItem,hvBESi9OnCifTgOwlt3qxpsYm8o,UExUbld0Wk1UZUhyUEZzQzR0RHB6TUZlcnN6WV9FUWx2SC...,https://www.youtube.com/watch?v=h3rFAj0RPgY,h3rFAj0RPgY,2024-06-27T15:37:16Z,UC1c9AbJs9TxnqYFF3lk0ROg,"Wednesday, June 26, 2024 - City Council Regula...",,{'default': {'url': 'https://i.ytimg.com/vi/h3...,...,https://i.ytimg.com/vi/h3rFAj0RPgY/hqdefault.jpg,h3rFAj0RPgY,4731.0,79.0,158,3,0,NaN,I call this meeting to order roll call please ...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,youtube#playlistItem,akR2TUoyGylxIjTKYUZ4XLTpYMg,UExiSXJiblQ5SW02SDNRWHRvSThocFNxVGNmOVcteWw1VC...,https://www.youtube.com/watch?v=Rkcg4fzOklE,Rkcg4fzOklE,2019-08-21T15:58:28Z,UCHQ1qSl5buMzLn6ZZRQcCPg,BOS 01 17 18,BOS 01 17 18,{'default': {'url': 'https://i.ytimg.com/vi/Rk...,...,https://i.ytimg.com/vi/Rkcg4fzOklE/hqdefault.jpg,Rkcg4fzOklE,7198.0,120.0,17,0,0,0,[Music] this is our meeting I'm calling this m...,True
187,youtube#playlistItem,_fO5_8BYlG788dGtigj8_q6fhr0,UExiSXJiblQ5SW02SDNRWHRvSThocFNxVGNmOVcteWw1VC...,https://www.youtube.com/watch?v=bWVzlOwAArw,bWVzlOwAArw,2019-08-21T15:58:28Z,UCHQ1qSl5buMzLn6ZZRQcCPg,Yuma County Board of Supervisors meeting 01/03...,Yuma County Board of Supervisors meeting 01/03...,{'default': {'url': 'https://i.ytimg.com/vi/bW...,...,https://i.ytimg.com/vi/bWVzlOwAArw/hqdefault.jpg,bWVzlOwAArw,5643.0,94.0,25,0,0,0,[Music] morning everybody I hope everybody had...,True
188,youtube#playlistItem,g1nIAi0M_KBMQ4DpZJF8NpmgGyo,UExiSXJiblQ5SW02SDNRWHRvSThocFNxVGNmOVcteWw1VC...,https://www.youtube.com/watch?v=fuWHuw9BRVc,fuWHuw9BRVc,2019-08-21T15:58:28Z,UCHQ1qSl5buMzLn6ZZRQcCPg,Yuma County Board of Supervisors Meeting 12/18/17,,{'default': {'url': 'https://i.ytimg.com/vi/fu...,...,https://i.ytimg.com/vi/fuWHuw9BRVc/hqdefault.jpg,fuWHuw9BRVc,8640.0,144.0,55,0,0,0,[Music] this is the regular meeting of the the...,True
189,youtube#playlistItem,5VJNQ8rqFFzp4woKfXE2tT0asCw,UExiSXJiblQ5SW02SDNRWH

In [44]:
FINAL_PLAYLIST_MERGE.shape[0]

191

In [55]:
column_rename_map_playlist = {

  "channelId": 'channel_id',
  "channelTitle":  'channel_title',
  "videoId": 'vid_id', 
  "title": 'vid_title',
  "description": 'vid_desc',
  "duration_approx_minutes": 'vid_length_min_approx',
  "duration_seconds": "duration_seconds",
  'publishedAt': 'vid_upload_date',
  'liveBroadcastContent': 'vid_livestreamed',
  'viewCount':  'vid_views',
  'likeCount': 'vid_likes',
  "commentCount": 'vid_comments',
  "favoriteCount" :'vid_favorites',
  "captions_raw": 'caption_text_raw',
  "caption_text_flag": "caption_text_flag",
  "meeting_date": "meeting_date",
  "video_link": "video_link",
  "thumbail": "thumbnail",
  
    }

In [56]:
renamed_cols_df = FINAL_PLAYLIST_MERGE.rename(columns=column_rename_map_playlist)




In [59]:

(
    renamed_cols_df[[c for c in column_rename_map_playlist.values() if c in renamed_cols_df.columns]]
).to_csv("../outputs/sample-transcripts-from-playlists.csv", index=False)